<h1>Data Sci Eng Methods</h1>
<h3>Assignment 1 – ML Data Cleaning and Feature Selection</h3>
<h4>Data Set</h4>
<a href="https://www.kaggle.com/datasets/rohanrao/nifty50-stock-market-data">NIFTY-50 Stock Market Data (2000 - 2021)</a>
<h5>About Dataset</h5>
<p>The data is the price history and trading volumes of the fifty stocks in the index NIFTY 50 from NSE (National Stock Exchange) India. All datasets are at a day-level with pricing and trading values split across .cvs files for each stock along with a metadata file with some macro-information about the stocks itself. The data spans from 1st January, 2000 to 30th April, 2021.</p>
Note: The dataset contains stock data for 50 stocks

<h5>Columns</h5>

Date - Trade Data

Symbol - Name of stock

Series - Type of security

Prev Close - Previous data closing price

Open - Opening price for the day

High - Highest price for the day

Low - Lowest price for the day

Last - Last trade price

Closes - Closing price

VWAP - Volume-weighted average price (a ratio of the cumulative share price to the cumulative volume traded over a given time period)

Volume - volume trades for the day

Turnover - The turnover ratio is ratio of sellers to buyers of a stock

Trades - Number of Trades

Deliverable Volume - Amount of deliverable volume

%Deliverble - Percentage of shares that were delivered

Note: All price are in Rupees

<h3>Aim:</h3>

* What are the data types? (Only numeric and categorical)

* Are there missing values?

* What are the likely distributions of the numeric variables?

* Which independent variables are useful to predict a target (dependent variable)? (Use at least three methods)

* Which independent variables have missing data? How much?

* Do the training and test sets have the same data?

* In the predictor variables independent of all the other predictor variables?

* Which predictor variables are the most important?

* Do the ranges of the predictor variables make sense?

* What are the distributions of the predictor variables?   

* Remove outliers and keep outliers (does if have an effect of the final predictive model)?

* Remove 1%, 5%, and 10% of your data randomly and impute the values back using at least 3 imputation methods. How well did the methods recover the missing values?  That is remove some data, check the % error on residuals for numeric data and check for bias and variance of the error.
the distributions of the predictor variables?


In [268]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import SelectKBest, f_classif, chi2, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [269]:
url = "https://raw.githubusercontent.com/ChaudharyAnshul/INFO6105_Assignments/main/Nifty50_Data/NIFTY50_all.csv"
data = pd.read_csv(url)

In [270]:
data.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [271]:
#Print all the Unique stock names
symbol = data["Symbol"].unique()
print(symbol)
print("Unique stock values: ", len(symbol))

['MUNDRAPORT' 'ADANIPORTS' 'ASIANPAINT' 'UTIBANK' 'AXISBANK' 'BAJAJ-AUTO'
 'BAJAJFINSV' 'BAJAUTOFIN' 'BAJFINANCE' 'BHARTI' 'BHARTIARTL' 'BPCL'
 'BRITANNIA' 'CIPLA' 'COALINDIA' 'DRREDDY' 'EICHERMOT' 'GAIL' 'GRASIM'
 'HCLTECH' 'HDFC' 'HDFCBANK' 'HEROHONDA' 'HEROMOTOCO' 'HINDALC0'
 'HINDALCO' 'HINDLEVER' 'HINDUNILVR' 'ICICIBANK' 'INDUSINDBK' 'INFOSYSTCH'
 'INFY' 'IOC' 'ITC' 'JSWSTL' 'JSWSTEEL' 'KOTAKMAH' 'KOTAKBANK' 'LT' 'M&M'
 'MARUTI' 'NESTLEIND' 'NTPC' 'ONGC' 'POWERGRID' 'RELIANCE' 'SBIN'
 'SHREECEM' 'SUNPHARMA' 'TELCO' 'TATAMOTORS' 'TISCO' 'TATASTEEL' 'TCS'
 'TECHM' 'TITAN' 'ULTRACEMCO' 'UNIPHOS' 'UPL' 'SESAGOA' 'SSLT' 'VEDL'
 'WIPRO' 'ZEETELE' 'ZEEL']
Unique stock values:  65


<h5>Some of the stock are renamed over time</h5>

MUNDRAPORT -> ADANIPORTS

BAJAUTOFIN -> BAJFINANCE

BHARTI -> BHARTIARTL

HINDLEVER -> HINDUNILVR

JSWSTL -> JSWSTEEL

KOTAKMAH -> KOTAKBANK

TELCO -> TATAMOTORS

TISCO -> TATASTEEL

UNIPHOS -> UPL

SESAGOA -> SESAGOA -> VEDL

HINDALC0 -> HINDALCO

In [272]:
#replacing the values
replace = [
    ["MUNDRAPORT","ADANIPORTS"],
    ["BAJAUTOFIN","BAJFINANCE"],
    ["BHARTI","BHARTIARTL"],
    ["HINDLEVER","HINDUNILVR"],
    ["JSWSTL","JSWSTEEL"],
    ["KOTAKMAH","KOTAKBANK"],
    ["TELCO","TATAMOTORS"],
    ["TISCO","TATASTEEL"],
    ["SESAGOA","VEDL"],
    ["SESAGOA","VEDL"],
    ["HINDALC0","HINDALCO"],
    ["UNIPHOS","UPL"],
]
for i in replace:
    data.loc[ data["Symbol"] == i[0], "Symbol"] = i[1]

In [273]:
data["Symbol"].unique()

array(['ADANIPORTS', 'ASIANPAINT', 'UTIBANK', 'AXISBANK', 'BAJAJ-AUTO',
       'BAJAJFINSV', 'BAJFINANCE', 'BHARTIARTL', 'BPCL', 'BRITANNIA',
       'CIPLA', 'COALINDIA', 'DRREDDY', 'EICHERMOT', 'GAIL', 'GRASIM',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HEROHONDA', 'HEROMOTOCO',
       'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'INDUSINDBK', 'INFOSYSTCH',
       'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M',
       'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'POWERGRID', 'RELIANCE',
       'SBIN', 'SHREECEM', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'VEDL', 'SSLT', 'WIPRO',
       'ZEETELE', 'ZEEL'], dtype=object)

In [274]:
# the data can be categorized based on the "Symbol" since it contains all the different possible stock
data = data.astype({'Symbol': 'category'})

In [275]:
# convert column date to type date
data['Date'] = pd.to_datetime(data['Date'])

In [276]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235192 entries, 0 to 235191
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Date                235192 non-null  datetime64[ns]
 1   Symbol              235192 non-null  category      
 2   Series              235192 non-null  object        
 3   Prev Close          235192 non-null  float64       
 4   Open                235192 non-null  float64       
 5   High                235192 non-null  float64       
 6   Low                 235192 non-null  float64       
 7   Last                235192 non-null  float64       
 8   Close               235192 non-null  float64       
 9   VWAP                235192 non-null  float64       
 10  Volume              235192 non-null  int64         
 11  Turnover            235192 non-null  float64       
 12  Trades              120344 non-null  float64       
 13  Deliverable Volume  219115 no

<h3>Q: What are the data types? (Only numeric and categorical)</h3>
A: the data types are listed below

* There are 11 columns  with numeric data 'Prev Close', 'Open', 'High', 'Low', 'Last',  'Close', 'VWAP', 'Volume', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble'.
* There is 1 category data 'Symbol'.
* There are 1 datetime data 'Date'.

In [277]:
# finding null values if any
data.isnull().sum()

Date                       0
Symbol                     0
Series                     0
Prev Close                 0
Open                       0
High                       0
Low                        0
Last                       0
Close                      0
VWAP                       0
Volume                     0
Turnover                   0
Trades                114848
Deliverable Volume     16077
%Deliverble            16077
dtype: int64

<h3>Q: Are there missing values?</h3>
A: There are NaN values present in the data, the dataset captured 0 as Null/NaN hence filling the null values with 0 for Trades, Deliverable Volume and %Deliverble.

Explained below:

<b>Number of Trades</b> can be 0 - It's very simple to understand that if the deal which is comprised of sellers and buyers is zero, the buyers and sellers for that trade aren't buying or selling.
Volumes are always positive if not zero and can be represented as the number of trades per day for a commodity, stock, options, or future.

<b>Deliverable Volume</b> can be 0 - In the context of stock trading and financial markets, "deliverable volume" typically refers to the number of shares of a particular stock that are traded and actually delivered to the buyer or seller during a trading session. It represents the number of shares that have changed ownership. There are some situations where deliverable volume might be very low, or zero for certain stocks.

<b>%Deliverble</b> can be 0 - It is possible for the "% Deliverable" (percentage of shares that were delivered) of a stock to be zero or close to zero on a particular trading day. "% Deliverable" is a measure that indicates the proportion of shares traded for a particular stock on a given day that were actually delivered to the buyers' accounts.


In [278]:
# replace null values with 0 based on the above explanation
data["Trades"].fillna(0, inplace = True)
data["Deliverable Volume"].fillna(0, inplace = True)
data["%Deliverble"].fillna(0, inplace = True)

In [279]:
# checking if there is still any null values
data.isnull().sum()

Date                  0
Symbol                0
Series                0
Prev Close            0
Open                  0
High                  0
Low                   0
Last                  0
Close                 0
VWAP                  0
Volume                0
Turnover              0
Trades                0
Deliverable Volume    0
%Deliverble           0
dtype: int64

<h3>Q: Which independent variables have missing data? How much?</h3>
A: There are no missing data present in the dataset, since in the above code the null values are already handled.

In [ ]:
# @title Checking distrubution of the data
# density distrubution for entire data set
plt.subplot(231)
sns.histplot(data['Prev Close'], kde=True, stat="density", linewidth=0)

plt.subplot(232)
sns.histplot(data['Open'], kde=True, stat="density", linewidth=0)

plt.subplot(233)
sns.histplot(data['High'], kde=True, stat="density", linewidth=0)

plt.subplot(234)
sns.histplot(data['Low'], kde=True, stat="density", linewidth=0)

plt.subplot(235)
sns.histplot(data['Close'], kde=True, stat="density", linewidth=0)

plt.subplot(236)
sns.histplot(data['VWAP'], kde=True, stat="density", linewidth=0)
fig = plt.gcf()
fig.set_size_inches(10,10)

<h4>Inference:</h4>

* All the distributions show similar distribution properties
* All the distributions are right skewed, that means they are positively skewed

Need to select individual stocks and verify the skewness of data, since different socks trade in different price range.



In [ ]:
# Selecting individual Stocks to carry forward the analysis

# ADANIPORTS
data_ap = data.loc[ data["Symbol"] == "ADANIPORTS"]
# HDFCBANK
data_hb = data.loc[ data["Symbol"] == "HDFCBANK"]

In [ ]:
# Dropping Columns since they are not important
data_ap = data_ap.drop(labels=["Series","Symbol"], axis=1)
data_hb = data_hb.drop(labels=["Series","Symbol"], axis=1)

In [ ]:
# density distrubution for ADANIPORTS
plt.subplot(231)
sns.histplot(data_ap['Prev Close'], kde=True, stat="density", linewidth=0)

plt.subplot(232)
sns.histplot(data_ap['Open'], kde=True, stat="density", linewidth=0)

plt.subplot(233)
sns.histplot(data_ap['High'], kde=True, stat="density", linewidth=0)

plt.subplot(234)
sns.histplot(data_ap['Low'], kde=True, stat="density", linewidth=0)

plt.subplot(235)
sns.histplot(data_ap['Close'], kde=True, stat="density", linewidth=0)

plt.subplot(236)
sns.histplot(data_ap['VWAP'], kde=True, stat="density", linewidth=0)
fig = plt.gcf()
fig.set_size_inches(10,10)

<h5>Similar to above Inference</h5>

* All the distributions show similar distribution properties
* All the distributions are right skewed showing positive skewness


In [ ]:
# density distrubution for HDFCBANK
plt.subplot(231)
sns.histplot(data_hb['Prev Close'], kde=True, stat="density", linewidth=0)

plt.subplot(232)
sns.histplot(data_hb['Open'], kde=True, stat="density", linewidth=0)

plt.subplot(233)
sns.histplot(data_hb['High'], kde=True, stat="density", linewidth=0)

plt.subplot(234)
sns.histplot(data_hb['Low'], kde=True, stat="density", linewidth=0)

plt.subplot(235)
sns.histplot(data_hb['Close'], kde=True, stat="density", linewidth=0)

plt.subplot(236)
sns.histplot(data_hb['VWAP'], kde=True, stat="density", linewidth=0)
fig = plt.gcf()
fig.set_size_inches(10,10)

<h5>Inference:</h4>

* All the distributions show similar distribution properties
* All the distributions are right skewed showing positive skewness



<h3>Q: What are the likely distributions of the numeric variables?</h3>

A: For the above density distributions it is observed that the distributions are right-skewed.


<h3>Q: What are the distributions of the predictor variables?</h3>
A: Predictor variables are identified below they are - ["Prev Close", "Open", "High", "Low", Last", "VWAP"]. As per the graph above we can see that the predictors are skewed towards right resulting in Positively skewed (right-skewed) distributions.


In [ ]:
# Checking the Ranges of the predictor variables and dependent variables for ADANIPORTS
plt.figure(figsize=(20, 7))
sns.boxplot(data=data_ap[["Prev Close", "Open", "High", "Low", "Close", "VWAP"]])

<h5>Inference:</h5>

Looking at the box plot above it can be inferred that the values are in the same range and there are outliers present in all the columns. Considering the fact that the data is for a stock it is possible that the price would have shot up which explains the outliers exceeding the maximum value."


In [ ]:
# Checking the Ranges of the predictor variables and dependent variables for HDFCBANKHDFCBANK
plt.figure(figsize=(20, 7))
sns.boxplot(data=data_hb[["Prev Close", "Open", "High", "Low", "Close", "VWAP"]])

<h4>Inference:</h4>

Looking at the box plot above it can be inferred that the values are in the same range and no data transformation is required.


<h3>Q: Do the ranges of the predictor variables make sense?</h3>
A: The range of dataset makes sense as all the variables have similar Min and Max. The values are within expected and reasonable range based on the nature of the variable.


<h4>Feature Selection Methods</h4>

1. Correlation Coefficient
2. SelectKBest
3. f_regression


Target variable is "Close"

<h4>Method 1: Correlation Coefficient</h4>

In [ ]:
# correlation matrix for Asianpaints
corr = data_ap.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap="coolwarm")

In [ ]:
# correlation matrix for HDFCBank
corr = data_hb.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr, annot=True, cmap="coolwarm")

<h4>Inference:</h4>
Based on the above matrix "Prev Close", "Open", "High", "Low", Last", "VWAP" are strongly correlated with the target variable. This indicates a significant relationship with the target.



In [ ]:
# spliting the dependent and independent variables
X_ap,y_ap = data_ap.drop(labels=["Date","Close"], axis=1), data_ap["Close"]
X_hb,y_hb = data_hb.drop(labels=["Date","Close"], axis=1), data_hb["Close"]

<h4>Method 2: SelectKBest</h4>

In [ ]:
# selectKBest method
# trying all possible values of K to get an idea of the most impotant variables
for i in range(2,12):
    selector_1 = SelectKBest(score_func=f_classif, k=i)
    selector_2 = SelectKBest(score_func=f_classif, k=i)

    selector_1.fit_transform(X_ap, y_ap)
    selected_features_mask_1 = selector_1.get_support()

    selector_2.fit_transform(X_hb, y_hb)
    selected_features_mask_2 = selector_2.get_support()

    print("for AsianPaints K = ",i)
    print("selected values: ", [v for v, b in zip(X_hb.columns, selected_features_mask_1) if b == True ])

    print("for HDFCBank K = ",i)
    print("selected values: ", [v for v, b in zip(X_hb.columns, selected_features_mask_1) if b == True ])


<h5>This shows based on the k values, all independent variables that affect the dependent variables</h5>

<h4>Method 3: f_regression</h4>

In [ ]:
#f_regression method to identify all important variables
f_scores_ap, p_values_ap = f_regression(X_ap, y_ap)

print("f-scores and p-values for Asianpaints")
print("F-scores:")
print(list(zip(X_ap.columns, f_scores_ap)))
print("p-values:")
print(list(zip(X_ap.columns, p_values_ap)))

f_scores_hb, p_values_hb = f_regression(X_hb, y_hb)

print("\n\nf-scores and p-values for HDFCBank")
print("F-scores:")
print(list(zip(X_hb.columns, f_scores_hb)))
print("p-values:")
print(list(zip(X_hb.columns, p_values_hb)))

<h5>Smaller p-values indicate a stronger relationship between the feature and the target variable</h5>

<h3>Q: Which independent variables are useful to predict a target (dependent variable)? (Use at least three methods)</h3>
A: Based on the 3 methods shown above it can observe that

* heatmap: there are 6 independent variables that are more useful in prediction "Prev Close", "Open", "High", "Low", Last", "VWAP".

* SelectKBest: it can be observed that as we keep increasing the K value the variables based on importance are increased with K. We get the priority order of the features, but we still need some more information to find the main features.

* f_regression: based on the p-values of the independent variables it can be observed that 6 features have p-value as 0, ('Prev Close', 0.0), ('Open', 0.0), ('High', 0.0), ('Low', 0.0), ('Last', 0.0), ('VWAP', 0.0)

we can conclude based on the 3 results that 6 features are important in the prediction of Close Price.



In [ ]:
# Pair plot to find relation between independent variables
sns.pairplot(X_ap)
plt.show()

From the plot above it can be observed that "Prev Close", "Open", "High", "Low", Last", "VWAP" are highly correlated to eachother




<h3>Q: In the predictor variables independent of all the other predictor variables?</h3>
A: From the heatmap and pair plot above we can observe that the predictor variables are Multicollinearity.

Predictor: "Prev Close", "Open", "High", "Low", Last", "VWAP".



<h3>Q: Which predictor variables are the most important?</h3>
A: Based on the feature selection above following are the most imporant

* "Prev Close"
* "Open"
* "High"
* "Low"
* "Last"
* "VWAP"

In [ ]:
# split the data into training and testing for Asianpaints
X_train_ap, X_test_ap, y_train_ap, y_test_ap = train_test_split(X_ap, y_ap, test_size=0.2, random_state=42)

# split the data into training and testing for HDFCBank
X_train_hb, X_test_hb, y_train_hb, y_test_hb = train_test_split(X_hb, y_hb, test_size=0.2, random_state=42)

In [ ]:
#Plotting the data to see the histogram for Asianpaints for Test and Train data
for c in X_test_ap.columns:
    plt.hist(X_train_ap[c], bins=10, alpha=0.5, label="Train Set", color="darkgreen")
    plt.hist(X_test_ap[c], bins=10, alpha=0.5, label="Test Set", color ="red")
    plt.xlabel(c, size=14)
    plt.ylabel("Count", size=14)
    plt.legend(loc='upper right')
    plt.title("{} distribution".format(c))
    plt.show()

<h5>Inference:</h5>

* By looking at the split of data it looks like data is properly distributed in the ratio of 80%:20% for train and test respectively
* it looks like as we move towards the right on x-axis the split of train:test decreases.

In [ ]:
#Plotting the data to see the histogram for HDFCBank for test and train data
for c in X_test_hb.columns:
    plt.hist(X_train_hb[c], bins=10, alpha=0.5, label="Train Set", color="darkgreen")
    plt.hist(X_test_hb[c], bins=10, alpha=0.5, label="Test Set", color ="red")
    plt.xlabel(c, size=14)
    plt.ylabel("Count", size=14)
    plt.legend(loc='upper right')
    plt.title("{} distribution".format(c))
    plt.show()

<h5>Inference:</h5>

* By looking at the split of data it looks like data is properly distributed in the ratio of 80%:20% for train and test respectively
* It looks like from the above split the data is split uniformly across all values.

<h3>Q: Do the training and test sets have the same data?</h3>
A: No, the test and train data dont have same data values. This is the stock price for each day so the chance of the open, close, high, low and vwap combined to be same between 2 days is nearing 0.

<h3>Linear regression Model</h3>

In [ ]:
# Create a scatter plot with the data points for Asianpaints
for i in X_ap:
    plt.figure(figsize=(7, 7))
    sns.scatterplot(x=X_ap[i], y=y_ap, label='Data Points')

    # Fit a linear regression line to the data
    sns.regplot(x=X_ap[i], y=y_ap, ci=None, label='Linear Regression Line')
    plt.xlabel(i)
    plt.ylabel('Close')
    plt.title('Linear Regression Plot')
    plt.legend()
    plt.grid(True)
    plt.show()

<h4>Inference:</h4>
It can be observed that the 6 features we selected from feature selection show a linear relationship with the target variable



<h3>Model Training</h3>
Training a linear regression model for Asian paints and HDFCBank
<h4>Training the model with features that impact the target value the most</h4>



In [ ]:
# keep relavent columns after feature selection
X_train_ap, X_test_ap = X_train_ap[["Prev Close", "Open", "High", "Low", "Last", "VWAP"]], X_test_ap[["Prev Close", "Open", "High", "Low", "Last", "VWAP"]]
X_train_hb, X_test_hb = X_train_hb[["Prev Close", "Open", "High", "Low", "Last", "VWAP"]], X_test_hb[["Prev Close", "Open", "High", "Low", "Last", "VWAP"]]

In [ ]:
# model for Asianpaints
model_ap = LinearRegression()
# model for HDFCBank
model_hb = LinearRegression()

In [ ]:
#fit the model with Asianpaints data
model_ap.fit(X_train_ap, y_train_ap)

In [ ]:
#fit the model with HDFCBank data
model_hb.fit(X_train_hb, y_train_hb)

In [ ]:
# predict Asianpaint
y_pred_ap = model_ap.predict(X_test_ap)

In [ ]:
r2_ap = r2_score(y_test_ap, y_pred_ap)
print("R-squared:", r2_ap)

In [ ]:
# predict HDFCBank
y_pred_hb = model_hb.predict(X_test_hb)

In [ ]:
r2_hb = r2_score(y_test_hb, y_pred_hb)
print("R-squared:", r2_hb)

<h4>Inference:</h4>

* The R-squared value is nearing 1 in both the cases
* R-squared metric helps understand that the regression model is a good fit to the data,


In [ ]:
# Mean Squared Error for Asianpaint model
mse_ap = mean_squared_error(y_test_ap, y_pred_ap)
print("Mean of Asian Paint data:", y_test_ap.mean())
print("Mean Squared Error:", mse_ap)

In [ ]:
# Mean Squared Error for HDFCBank model
mse_hb = mean_squared_error(y_test_hb, y_pred_hb)
print("Mean of HDFC Bank data:", y_test_hb.mean())
print("Mean Squared Error:", mse_hb)

<h4>Inference:</h4>

* The mean squared error of both the model is significantly lower than their respected mean values indicating a good fit of model



<h4>Removing Outrliers</h4>

In [ ]:
# data cleaning for Asianpaints
# taking the z-score to find the outliers
z_scores_ap = stats.zscore(y_train_ap)

In [ ]:
# box plot to check the threshold value
sns.boxplot(data = z_scores_ap.to_frame())

The values above 2 are the outliers in the data

In [ ]:
threshold = 2

In [ ]:
# Drop rows based on the threshold value
X_train_ap_clean = X_train_ap.copy() # make copy
X_train_ap_clean["Zscore"] = z_scores_ap # add column
X_train_ap_clean["Close"] = y_train_ap # add column

index_ap = X_train_ap_clean[ (X_train_ap_clean['Zscore'] > threshold) ].index # find the index to drop
X_train_ap_clean.drop(index_ap , inplace=True) # drop rows

# split features
y_train_ap_clean = X_train_ap_clean["Close"]
X_train_ap_clean = X_train_ap_clean.drop(["Zscore", "Close"], axis=1)

In [ ]:
# data Cleaning for  HDFCBank
# taking the z-score to find the outliers
z_scores_hb = stats.zscore(y_train_hb)

In [ ]:
# box plot to check the threshold valu
sns.boxplot(data=z_scores_hb.to_frame())

<h5>As we can observe from the plot there are no outliers present in the data for HDFCBank</h5>

Proceeding with training the model with clean data for Asianpaints

In [ ]:
# model for Asianpaints
model_ap_new = LinearRegression()

In [ ]:
# fit the model with HDFCBank data
model_ap_new.fit(X_train_ap_clean, y_train_ap_clean)

In [ ]:
# predict Asianpaint
y_pred_ap_new = model_ap_new.predict(X_test_ap)

In [ ]:
r2_ap_new = r2_score(y_test_ap, y_pred_ap_new)
print("R-squared:", r2_ap_new)

In [ ]:
# Mean Squared Error for Asianpaint model
mse_ap_new = mean_squared_error(y_test_ap, y_pred_ap_new)
print("Mean of Asian Paint data:", y_test_ap.mean())
print("Mean Squared Error:", mse_ap_new)

<h3>Q: Remove outliers and keep outliers (does if have an effect of the final predictive model)?</h3>
A: After removing the outliers from the data there is not much change in the R-squared score of the predictive model.
i think this was because the number of outliers in the data were very small compared to the actual dataset

**<h5>Removeing 1% 5% and 10% of the data randomly from the dataset</h5>**

In [ ]:
original_data = data_ap["Open"] # original data
# to remove data from row
def RemoveData(per):
    remove_per = per
    num_rows = int(len(original_data) * remove_per) # number of row to delete
    rows_delete = np.random.choice(original_data.index, num_rows, replace=False) # choose randomly
    data_less = original_data.drop(rows_delete)  # drop random rows
    print("Number of rows to populate ", per*100, "% :",num_rows)
    return data_less, num_rows

In [ ]:
# to add data to row and calculate error
def FillDataAndCalculateError(data, value, num_rows, per):
    data_less_df = data.to_frame()  # convert to frame
    new_data = {
        'Open': [value] * num_rows,
    }
    new_df = pd.DataFrame(new_data)
    new_df = pd.concat([data_less_df, new_df]) # merge frame


    # calculating Error Bias and Variance
    residuals = original_data - new_df["Open"]
    percent_errors = np.abs(residuals / original_data) * 100
    bias = np.mean(percent_errors)
    variance = np.std(percent_errors)
    print("Error for %", per, " (Bias): ", bias)
    print("Error for %", per, " (Variance): ", variance)

In [ ]:
# removing 1% data
remove_per = 0.01
data_less_1, num_rows_1 = RemoveData(remove_per)

# removing 5% data
remove_per = 0.05
data_less_5, num_rows_5 = RemoveData(remove_per)

# removing 10% data
remove_per = 0.1
data_less_10, num_rows_10 = RemoveData(remove_per)

#Added back mean of all the values
FillDataAndCalculateError(data_less_1, float(data_less_1.to_frame().mean()), num_rows_1, 1)
FillDataAndCalculateError(data_less_5, float(data_less_5.to_frame().mean()), num_rows_5, 5)
FillDataAndCalculateError(data_less_10, float(data_less_10.to_frame().mean()), num_rows_10, 10)

In [ ]:
# removing 1% data
remove_per = 0.01
data_less_1, num_rows_1 = RemoveData(remove_per)

# removing 5% data
remove_per = 0.05
data_less_5, num_rows_5 = RemoveData(remove_per)

# removing 10% data
remove_per = 0.1
data_less_10, num_rows_10 = RemoveData(remove_per)

#Added back median of all the values
FillDataAndCalculateError(data_less_1, float(data_less_1.to_frame().median()), num_rows_1, 1)
FillDataAndCalculateError(data_less_5, float(data_less_5.to_frame().median()), num_rows_5, 5)
FillDataAndCalculateError(data_less_10, float(data_less_10.to_frame().median()), num_rows_10, 10)

<h4>Inference</h4>
There is low Bias and low variance when the missing value is filled by mean, median but as the missing values increases the bias and variance also increases.



<h3>Q: Remove 1%, 5%, and 10% of your data randomly and impute the values back using at least 3 imputation methods. How well did the methods recover the missing values?  That is remove some data, check the % error on residuals for numeric data and check for bias and variance of the error.</h3>
A: The missing data was replaced by mean and median and it was observed that when the missing data percent is small bias and standard deviation of the error was small with increase in missing value the variance also increased


<h2>Conclusion:</h2>

*   The Above Dataset has both numeric and categorical data
*   After Feature Selection it was observed that the independent variables useful to predict are "Prev Close", "Open", "High", "Low", Last", "VWAP".
*   None of the Independent variables were missing in the dataset, however there were some missing values in the data which were replaced by 0
*   The Train and Test data are split in the ratio of 80%:20% and they don’t have the same values
*   The Predictor variables are Multicollinearity, i.e., they are highly related to each other
*   "Prev Close", "Open", "High", "Low", Last", "VWAP" are the most important predictor variables.
*   All the variables have similar Min and Max. The values are within expected and reasonable range based on the nature of the variable.
*   The Distribution of the predictor variables are right-skewed and have a positive skewed distribution.








<h2>References :</h2>


*   https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection
*   https://pandas.pydata.org/docs/reference/frame.html#dataframe
*   https://github.com/aiskunks/YouTube/tree/main/A_Crash_Course_in_Statistical_Learning/ML_Data_Cleaning_and_Feature_Selection
*   https://www.analyticsvidhya.com/blog/2020/10/feature-selection-techniques-in-machine-learning/
*   Stack Overflow to get around errors - https://stackoverflow.com
*   https://seaborn.pydata.org/
*   https://matplotlib.org/
*   https://github.com/nikbearbrown/





MIT License

Copyright (c) [2023] [Anshul Chaudhary]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS," WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

